In [1]:
import numpy as np
import xarray as xr
import xesmf as xe
import numpy as np
from mask_regions import mask_inland_seas, mask_poles

def get_lonlat_idx_nearest_to_tgs(tg_ds,ds):
    #tg_ds = xr.DataSet containing 'lon' and 'lat' coordinates of tide gauges
    #ds    = xr.DataSet containing CMIP6 data to subset
    
    lon_name = list(k for k in ds.dims if 'lon' in k)[0] #find lon/lat coordinate names
    lat_name = list(k for k in ds.dims if 'lat' in k)[0]
    
    #compute distances between TG coordinates and grid cell centers
    distances = 2*np.arcsin( np.sqrt(
        np.sin( (np.pi/180) * 0.5*(ds[lat_name]-tg_ds.lat) )**2 +
        np.cos((np.pi/180)*tg_ds.lat)*np.cos((np.pi/180)*ds[lat_name])*np.sin((np.pi/180)*0.5*(ds[lon_name]-tg_ds.lon))**2) )

    distances = distances.where(np.isfinite(ds),1e30)
    idx_nearest = distances.argmin(dim=[lon_name,lat_name]) #find indices of nearest grid cells
    return idx_nearest

Store data for analyzing the ratio of ASLC range changes to the historical tidal range:

In [2]:
#configure script
ssp='ssp370'
var='zos_ibe'

psmsl_fn = '../month_means_1993_2022_psmsl.nc'
EOT20_fn = '../EOT20_TR_MHD.nc'
sla_fn = 'gs://leap-persistent/timh37/allsat_l4_duacs_sla_1993-2023_0.125deg'
ibe_fn = 'gs://leap-persistent/timh37/monthly_ibe_1993-2023_0.125deg'

In [3]:
#tide-gauge data
psmsl = xr.open_dataset(psmsl_fn) #month means
psmsl = mask_poles(mask_inland_seas(psmsl))
psmsl = psmsl.where(np.isfinite(psmsl.isel(month=0,drop=True)),drop=True)
psmsl_range = 0.1*(psmsl.monmean.max(dim='month') - psmsl.monmean.min(dim='month'))#[cm]

psmsl_range.to_netcdf('aslc_tr/psmsl_mean_alsc_range.nc',mode='w')

#satellite altimetry and era5-based ibe (1/8 deg)
sla = xr.open_dataset(sla_fn,engine='zarr').rename({'latitude':'lat','longitude':'lon'})[['sla']].load()
ibe = xr.open_dataset(ibe_fn,engine='zarr').rename({'latitude':'lat','longitude':'lon'}).load()

sla = mask_poles(mask_inland_seas(sla))
ibe = mask_poles(mask_inland_seas(ibe))

sla_ibe = sla['sla']+ ibe['ibe'].values

sla_ibe_month_means = sla_ibe.groupby(sla_ibe.time.dt.month).mean()
sla_ibe_range = 100*(sla_ibe_month_means.max(dim='month') - sla_ibe_month_means.min(dim='month')) #[cm]
sla_ibe_range.to_netcdf('aslc_tr/allsat_l4_duacs_sla_1993-2023_0.125deg_mean_alsc_range.nc',mode='w')

Preprocess/open CMIP6 data:

In [4]:
from analysis_io import open_aslc_range

exclude_models = ['INM-CM5-0','INM-CM4-8','GISS-E2-1-G','MIROC-ES2L'] #poorly performing models

aslc_range = open_aslc_range('gs://leap-persistent/timh37/CMIP6/aslc_1x1',{'lat':500,'lon':500,'period':1,'member_id':50,'experiment_id':4}).sel(experiment_id=ssp).drop_sel(source_id=exclude_models)#.load()
aslc_range_tgs = open_aslc_range('gs://leap-persistent/timh37/CMIP6/aslc_psmsl',{'tg':1000,'period':2,'member_id':200}).sel(experiment_id=ssp).drop_sel(source_id=exclude_models).load()

more_than_10_models = np.isfinite(aslc_range.zos_ibe).any(dim='member_id').sum(dim='source_id')>=10 #mask for gridded fields where less than 10 models have data

aslc_range = mask_poles( mask_inland_seas( aslc_range.where(more_than_10_models) ) )
aslc_range_tgs = mask_poles( mask_inland_seas( aslc_range_tgs ) )

hist_range = aslc_range[var].isel(period=0).mean(dim='member_id').median(dim='source_id').load()
hist_range_tgs = aslc_range_tgs[var].isel(period=0).mean(dim='member_id').median(dim='source_id').load()

drange = (aslc_range[var].isel(period=1) - aslc_range[var].isel(period=0)).mean(dim='member_id').median(dim='source_id').load()
drange_tgs = (aslc_range_tgs[var].isel(period=1) - aslc_range_tgs[var].isel(period=0)).mean(dim='member_id').median(dim='source_id').load()

hist_range_tgs = hist_range_tgs.sel({'id':psmsl.id}) #select cmip6 data only where observations are not masked out 
drange_tgs = drange_tgs.sel({'id':psmsl.id}) #select cmip6 data only where observations are not masked out 

#store locally
hist_range.to_netcdf('aslc_tr/hist_aslc_range_memmean_median.nc',mode='w')
hist_range_tgs.to_netcdf('aslc_tr/hist_aslc_range_memmean_median_tgs.nc',mode='w')
drange.to_netcdf('aslc_tr/d_aslc_range_memmean_median_'+ssp+'.nc',mode='w')
drange_tgs.to_netcdf('aslc_tr/d_aslc_range_memmean_median_'+ssp+'_tgs.nc',mode='w')

#interpolate low-resolution cmip6 projections to eot20/era5 grid for plotting
regridder=xe.Regridder(drange,sla_ibe_range,'bilinear',periodic=True,ignore_degenerate=True)
drange_at_duacs = regridder(drange)
drange_at_duacs.to_dataset(name='zos_ibe').to_netcdf('aslc_tr/d_aslc_range_memmean_median_'+ssp+'_at_duacs.nc',mode='w')

hist_range_at_duacs = regridder(hist_range)
hist_range_at_duacs.to_dataset(name='zos_ibe').to_netcdf('aslc_tr/hist_aslc_range_memmean_median_at_duacs.nc',mode='w')

/srv/conda/envs/notebook/lib/python3.12/site-packages/zarr/core/metadata/v2.py:192: UserWarning: Found an empty list of filters in the array metadata document. This is contrary to the Zarr V2 specification, and will cause an error in the future. Use None (or Null in a JSON document) instead of an empty list of filters.
  warnings.warn(msg, UserWarning, stacklevel=1)
/srv/conda/envs/notebook/lib/python3.12/site-packages/zarr/core/metadata/v2.py:192: UserWarning: Found an empty list of filters in the array metadata document. This is contrary to the Zarr V2 specification, and will cause an error in the future. Use None (or Null in a JSON document) instead of an empty list of filters.
  warnings.warn(msg, UserWarning, stacklevel=1)
/srv/conda/envs/notebook/lib/python3.12/site-packages/zarr/core/metadata/v2.py:192: UserWarning: Found an empty list of filters in the array metadata document. This is contrary to the Zarr V2 specification, and will cause an error in the future. Use None (or Nul

Open & regrid EOT20 tidal range data:

In [5]:
tr = xr.open_dataset(EOT20_fn).load()
tr['tidal_range'] = tr.tidal_range*100 #convert m to cm

regridder = xe.Regridder(tr,sla_ibe_range,method='bilinear',periodic=True,ignore_degenerate=True) #regrid to altimetry grid
tr_at_duacs = regridder(tr)
tr_at_duacs.to_netcdf('aslc_tr/eot20_tr_at_duacs.nc',mode='w')

In [6]:
idx_nearest = get_lonlat_idx_nearest_to_tgs(hist_range_tgs,tr.tidal_range) #^not completely appropriate to first regrid then extrapolate, but ok for now?
    
tr_at_tgs = tr.isel(lon=idx_nearest['lon'],lat=idx_nearest['lat'])

tr_at_tgs['lat'] = ('id',hist_range_tgs.lat.values)
tr_at_tgs['lon'] = ('id',hist_range_tgs.lon.values)
tr_at_tgs.to_netcdf('aslc_tr/eot20_tr_at_psmsl.nc',mode='w')